In [56]:
from importlib.metadata import version
from flask import Flask, render_template_string, jsonify, request, redirect, render_template
import random
import json
import os
# 获取 requests 库的版本
print(version("os"))

PackageNotFoundError: No package metadata was found for os

In [51]:
import json
import re
from docx import Document

# -----------------------------
# 新增：读取 .docx 文件并保留“平”“仄”的加粗信息标记
# 使用特殊标记如 <b>平</b> 来记录加粗
# -----------------------------
def read_docx_text_with_format(file_path):
    doc = Document(file_path)
    full_text = []

    for para in doc.paragraphs:
        para_text = ""
        for run in para.runs:
            text = run.text
            # 判断是否加粗
            if run.bold:
                # 替换加粗的“平”“仄”为带标签版本
                text = text.replace('平', '<b>平</b>')
                text = text.replace('仄', '<b>仄</b>')
            para_text += text
        full_text.append(para_text)

    return '\n'.join(full_text)

# -----------------------------
# 修改：平仄转符号，支持加粗标记
# -----------------------------
def convert_tone_to_symbols(pattern):
    # 先处理加粗的平仄
    pattern = pattern.replace('<b>平</b>', '△')   # 加粗平 → 空心三角
    pattern = pattern.replace('<b>仄</b>', '▲')   # 加粗仄 → 实心三角
    # 再处理普通平仄
    pattern = pattern.replace('(平)', '◎')
    pattern = pattern.replace('(仄)', '◉')
    pattern = pattern.replace('平', '○')
    pattern = pattern.replace('仄', '●')
    return pattern

# -----------------------------
# 用符号长度计算字数（最准确）
# -----------------------------
def count_chars_from_symbols(s):
    symbol_str = convert_tone_to_symbols(s)
    chars = re.sub(r'[^○●◎◉△▲]', '', symbol_str)
    return len(chars)

# -----------------------------
# 按顺序块解析，杜绝错位
# -----------------------------
def parse_multiple_ci_pai(text):
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    all_ci = {}
    i = 0
    n = len(lines)

    while i < n:
        if lines[i] == "词牌名" and i + 1 < n:
            ci_name = lines[i + 1].strip("《》")
            i += 2  # 移动到下一行

            # 初始化字段
            description = ""
            tone_pattern_line = ""
            example_author = ""
            example_text = ""

            # 开始顺序读取后续字段
            while i < n:
                if lines[i] == "描述" and i + 1 < n:
                    description = lines[i + 1]
                    i += 2
                elif lines[i] == "词牌格式" and i + 1 < n:
                    tone_pattern_line = lines[i + 1]
                    i += 2
                elif lines[i] == "例词" and i + 2 < n:
                    example_author = lines[i + 1]
                    example_text = lines[i + 2]
                    i += 3
                elif lines[i] == "词牌名":
                    # 遇到下一个词牌名，跳出，开始新词牌
                    break
                else:
                    i += 1  # 跳过无关行（如空行或标题）
            # --- 字段解析结束 ---

            # 验证必要字段
            if not tone_pattern_line:
                print(f"⚠️ 跳过词牌 '{ci_name}'：缺少词牌格式")
                continue

            # 分句
            sentences = re.split(r'[，。]', tone_pattern_line)
            sentences = [s.strip() for s in sentences if s.strip()]

            # 计算每句字数
            sections = [{"chars": count_chars_from_symbols(s)} for s in sentences]
            total_chars = sum(item["chars"] for item in sections)

            # 转换平仄为符号（包含加粗符号）
            tone_pattern = [convert_tone_to_symbols(s) for s in sentences]

            # 构建 example
            example = {
                "title": ci_name,
                "author": example_author,
                "text": example_text
            }

            # 存入结果
            all_ci[ci_name] = {
                "description": description,
                "sections": sections,
                "tone_pattern": tone_pattern,
                "total_chars": total_chars,
                "example": example
            }
        else:
            i += 1  # 继续寻找“词牌名”

    return all_ci

# ⚠️ 修改为你的文件路径
file_path = '词牌名格式大全.docx'  # 必须是 .docx

try:
    if file_path.endswith('.docx'):
        text = read_docx_text_with_format(file_path)
    else:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()

    print("📄 文件读取成功，预览：\n")
    print(text[:500] + "...\n")

    result = parse_multiple_ci_pai(text)

    if not result:
        print("❌ 未解析到任何词牌，请检查格式是否包含“词牌名”等关键词。")
    else:
        print(f"✅ 成功解析 {len(result)} 个词牌：{list(result.keys())}")

        # 保存 JSON
        output_file = 'cipai.json'
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(result, f, ensure_ascii=False, indent=2)

        print(f"📁 已保存为：{output_file}")
        print("\n--- JSON 预览 ---")
        print(json.dumps(result, ensure_ascii=False, indent=2))

except Exception as e:
    print(f"❌ 错误：{e}")
    print("常见问题：")
    print("  - 文件路径错误")
    print("  - Word 段落分行不清晰")
    print("  - 缺少‘词牌名’、‘描述’、‘词牌格式’、‘例词’等关键词")
    print("  - ‘平’‘仄’未正确加粗（需用 Word 加粗，不能靠样式）")

📄 文件读取成功，预览：

词牌名
《十六字令》
描述
又被称作《苍梧谣》、《归字谣》，单调，十六字，三平韵。
词牌格式
<b>平</b>。(仄)仄平平仄仄<b>平</b>。平平仄，(仄)仄仄平<b>平</b>。
例词
毛泽东
山，快马加鞭未下鞍。惊回首，离天三尺三。山，倒海翻江卷巨澜。奔腾急，万马战犹酣。山，刺破青天锷未残。天欲堕，赖以拄其间。
词牌名
《捣练子》
描述
又被称作《捣练子令》，单调，廿七字，三平韵。
词牌格式
平仄仄，仄平<b>平</b>。(仄)仄平平(仄)仄<b>平</b>。(仄)仄(平)平平仄仄，(平)平(仄)仄仄平<b>平</b>。
例词
李煜
深院静，小庭空，断续寒砧断续风。无奈夜长人不寐，数声和月到帘栊。
词牌名
《忆江南》
描述
亦被称为：《江南好》，《望江南》，《梦江南》，《望江南》，《梦江口》， 《谢秋娘》，《春去也》，《归塞北》。单调二十七字，三平韵，北宋起开始有双调，实际不过是将单片重复而已。
词牌格式
平(平)仄，(仄)仄仄平<b>平</b>。(仄)仄(平)平平仄仄，(平)平(仄)仄仄平<b>平</b>。(仄)仄仄平<b>平</b>。
例词
白居易
江南好，风景旧曾谙。日出...

✅ 成功解析 50 个词牌：['十六字令', '捣练子', '忆江南', '忆王孙', '调笑令', '如梦令', '相见欢', '长相思', '生查子', '点绛唇', '浣溪沙', '菩萨蛮', '卜算子', '采桑子', '减字木兰花', '谒金门', '诉衷情', '忆秦娥', '清平乐', '更漏子', '阮郎归', '画堂春', '桃源忆故人', '摊破浣溪沙', '贺圣朝', '太常引', '西江月', '南歌子', '醉花阴', '浪淘沙', '鹧鸪天', '鹊桥仙', '虞美人', '南乡子', '玉楼春', '一斛珠', '踏莎行', '小重山', '蝶恋花其一', '蝶恋花其二', '一剪梅', '临江仙', '渔家傲', '唐多令', '苏幕遮', '定风波', '锦缠道', '谢池春', '青玉案', '天仙子']
📁 已保存为：cipai.json

--- JSON 预览 ---
{
  "十六字令": {
    "description": "又被称作《苍梧谣》、《归字谣》，单调，十六字，三平韵。",
    "s